## Time-series Synthetic Data

In [ ]:
def synthetic_experiment_performance_run_results(n1 = 50, n2 = 50, mean_M = 1, num_experiment=1, sigma = 0.1, pattern = 'block'):
    samples = np.zeros(num_experiment)
    t1 = time.time()

    algo_list = ['convex_debias', 'missing', 'OLS', 'PCA']

    if (pattern == 'block' or pattern == 'stagger'):
        algo_list.append('robust_synthetic_control')

    datas = np.zeros((num_experiment, len(algo_list)))

    #M0 = synthetic_M0(n1, n2, mean_M, 50, type='Gamma')

    E = np.random.normal(loc=0, scale=1, size=(n1, n2))

    E[:, 0] = 20
    M0 = np.cumsum(E, axis = 1)

    #M0 = M0 - np.mean(M0, axis = 1)
    #M0 = M0 * 10
    #M0 = readData.read_data('sales')
    #for i in range(2):
    #    M = synthetic_M0(n1, n2, mean_M/5, 30, type='Gaussian')
    #    M0 = M + M0
    s = np.linalg.svd(M0, full_matrices=False, compute_uv=False)
    plt.plot(s)
    plt.show()


    for T in range(num_experiment):
        if (T % 100 == 0):
            print(time.time() - t1)
            print('experiment ', T)

        #r = np.random.randint(low=1, high=10+1)
        #r = 5
        r = 10
        ## generating stagger pattern Z
        treat_units = []
        m1 = 0
        m2 = 0
        if (pattern == 'stagger'):
            m1 = np.random.randint(low=1, high=n1)
            m2 = np.random.randint(low=1, high=n2)
            Z = generate_Z(pattern_tuple=['stagger', (m1, m2)], M0=M0)
            treat_units = []

        if (pattern == 'block'):
            m1 = np.random.randint(low=1, high=int(n1/10))
            m2 = np.random.randint(low=1, high=int(n2/3))
            #m1 = 1
            Z, treat_units = generate_Z(pattern_tuple=['block', (m1, m2)], M0=M0)

        if (pattern == 'adaptive'):
            while True:
                a = np.random.randint(40)+5
                b = np.random.randint(40)+5
                #b = 2
                Z, info = generate_Z(pattern_tuple = ['adaptive', (a, b)], M0=M0)
                if (info == 'fail'):
                    continue
                break
        
        print('***sparsity****', np.sum(Z) / np.size(Z))


        tau_star = 1

        #PTperpZ = projection_T_orthogonal(Z, M0)

        #predict_sigma = sigma / np.sqrt(np.sum(PTperpZ**2))



        s = np.linalg.svd(M0, full_matrices=False, compute_uv=False)


        def test():
            O = M0 + Z * tau_star 
            r = 15
            suggest_l = s[r-1]/1.1

            results = run_algo(algo_list, O, Z, suggest_r = r, suggest_l = suggest_l, eps = 1e-4, de_mean_O=False, treat_units=treat_units, tau_star = tau_star, m2 = m2, M0 = M0)
            
            error_metric = {}
            for algo in algo_list:
                (M, tau) = results[algo]
                error_metric[algo] = metric_compute(M, tau, M0, tau_star, Z, ['tau_diff'])['tau_diff']
            return error_metric

        error_metric = test()
        print(error_metric)
        for index, algo in enumerate(algo_list):
                datas[T, index] = error_metric[algo]
        print('experiment {}, time elapses '.format(T), time.time() - t1)
    datas = pd.DataFrame(datas, columns = algo_list)
    return datas


pattern = 'stagger'
datas = synthetic_experiment_performance_run_results(n1 = n1, n2 = n2, mean_M = mean_M, sigma = sigma, num_experiment = 100, pattern = pattern)
file_name = 'results/plot_results/synthetic_{}_m1_m2_performance_results.p'.format(pattern)
#file_name = 'tmp.p'
pickle.dump(datas, open(file_name, 'wb'))